# Construction d'un système de recommandation

Nous avons décidé d'orienter notre projet sur la recommendation de films.
En effet durant ce confinement, nous avons eu le temps de visionner beaucoup de films,
mais nous nous sommes rendus compte que nous passions quasiment autant de temps
à choisir le film qu'à le regarder. D'où la nécessité de créer un système de re-
commendations afin d'optimiser notre temps de visionnage.
Nous avons chercher une base de données assez exploitable afin de mener à bien
notre projet. Nous nous sommes basés sur la base de données de 'The Movies Dataset'.

# Différents systèmes de recommandation

- [x] memory-based (user- et item- based)
- [ ] hybride
- [ ] model-based (matrix factorisation, optimisation avec descente de gradient)
- [x] popularity based = moyenne simple
- [ ] user-centered linear approach = descente de gradient (même pb d'opti que model-based)
- [ ] clustering (??)

In [459]:
import logging
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import re
from time import time
from ast import literal_eval

## Fetching and cleaning data

Nous utilisons deux tables de données. L'une, *movies_metadata.csv*, contient une liste de films et des informations relativesau genre, date de sortie etc. 

### Informations sur les films

In [206]:
movies = pd.read_csv("movies_metadata.csv")
movies.head()

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [207]:
movies.describe()

,revenue,runtime,vote_average,vote_count
count,4.546000e+04,45203.000000,45460.000000,45460.000000
mean,1.120935e+07,94.128199,5.618207,109.897338
std,6.433225e+07,38.407810,1.924216,491.310374
min,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,107.000000,6.800000,34.000000
max,2.787965e+09,1256.000000,10.000000,14075.000000


In [208]:
def filter_correct_id(word):
    if re.fullmatch(r'[0-9]+', word):
        return word
    return "wrong_id"

In [209]:
# don't re-run
movies = movies[~movies.id.duplicated()]
movies.id = movies.id.apply(filter_correct_id)
movies = movies[movies.id != "wrong_id"]
movies.id = movies.id.astype('int64')

### Avis des utilisateurs

In [210]:
ratings = pd.read_csv("ratings_small.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [211]:
# ne pas re-run !
ratings = ratings.drop(columns=['timestamp'])
ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [212]:
ratings[(ratings['userId'] == 1) & (ratings['movieId'] == 31)]

,userId,movieId,rating
0,1,31,2.5


In [213]:
print(min(ratings.rating), max(ratings.rating))
ratings.describe()
ratings.dtypes

0.5 5.0


userId       int64
movieId      int64
rating     float64
dtype: object

In [214]:
nbPers = len(ratings.userId.unique())
nbMovi = len(ratings.movieId.unique())

## Top films par genres

### Selection des genres

In [ ]:
movies = movies.rename(columns={'id' : 'movieId'})
movies = movies.loc[:, ['movieId', 'genres', 'title']]
movies.head()

In [ ]:
def simplify_genre(l):
    if len(l) <= 0 :
        return []
    if isinstance(l[0], dict):
        return [d['name'] for d in l]
    return l

movies['genres'] = movies['genres'].apply(lambda x: literal_eval(x) if isinstance(x, str) else x).apply(simplify_genre)
movies.head()

### Normaliser les notes sur toutes les données

In [ ]:
movies_small = movies.loc[movies['movieId'] <= 100]
movies_small['mean_rating'] = movies_small['movieId'].apply(lambda x: mean_base(ratings, x, 'movie'))

In [ ]:
movies_small.head()

In [ ]:
movies_small = movies_small.sort_values(by='mean_rating', ascending=False)

In [ ]:
movies_small.head()

In [ ]:
# 1 genre par ligne
serie = movies_small.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
serie.name = 'genres'
df_simpl = movies_small.drop('genres', axis=1).join(serie)
df_simpl.head()

In [ ]:
def top(df_simpl, genres):
    # selection des films par genre
    df_genres = df_simpl.loc[df_simpl['genres'].isin(genres)]
    df_genres = df_genres.sort_values(by='mean_rating', ascending=False)
    return df_genres

In [ ]:
top(df_simpl, ['Drama', 'Comedy']).head()

## Recommandation hybride

## Collaborative filtering : user- et item- based

Pour prédire la note d'un couple (*user*, *movie*) on peut regarder quelle note les utilisateurs similaires à *user* ont donné à ce film et faire une moyenne de leurs notes. On peut également regarder quelle note *user* a donné à des films similaires à *movie*. La première approche est centré sur les utilisateurs, *user-based*, tandis que la deuxième est centrée sur les films, *item-based*. Neánsmoins les deux approches suivent la même logique et nous allons implémenter des fonctions qui s'adaptent en fonction de l'approche choisie. Dans un système *user-based*, nous allons appeler **peers** les **users** et **others** les **items**. Dans un système *item-based* c'est l'inverse.



Deux utilisateurs sont considérés comme similaires s'ils ont les mêmes préférences de films. Il semble en effet plus pertinent de demander à un utilisateurs aux goûts similaires à *user* de lui conseiller un film. Pour comparer deux utilisateurs il faudra donc regarder les notes qu'ils ont donné aux mêmes films. De manière analogue, deux films sont similaires s'ils sont appréciés par les mêmes utilisateurs. Il faudra donc regarder les notes données par les mêmes utilisateurs pour comparer deux films. Cette notion de similitude sera calculée par un taux de corrélation.

Dans la suite du code, on considère l'existence d'une variable globale `df` contenant la dataframe des notes. Cela nous permet d'abord tester notre code sur des petits échantillons et avant de les faire tourner sur la totalité des donneés, sans avoir à modifier le code. De même, nous allons considérer une variable globale `cm_user` et `cm_movie` contenant la matrice de correlation entre utilisateurs et films. 

### Normalisation des notes

Nous n'avons besoin pour ce système que des notes données par les utilisateurs. Puisque la moyenne des notes données varie d'un utilisateur à un autre et d'un film à un autre, nous allons translater les notes afin que la moyenne des notes se trouve à 0. En *user-based*, on considère la moyenne par utilisateur, tandis qu'en *item-based* on s'interèsse à la moyenne par film. Par abus de langage nous appelons ces nouvelles notes les notes *normalisées*. 

In [348]:
def mean_rating(pid, base):
    '''
    Retourne la moyenne des notes données par l'utilisateur d'id uid
    '''
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    n = df.loc[ratings[ptype] == pid].count().loc[ptype]
    s = df.loc[ratings[ptype] == pid].sum().loc['rating']
    return s / n

In [365]:
def normalize(base='user'):
    '''
    Ajoute une colonne dans la dataframe df contenant les notes normalisées des utilisateurs
    '''
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    mean = df.loc[:, [ptype]].drop_duplicates()
    mean['mu'] = mean[ptype].map(lambda pid : mean_rating(pid, base))
    mean = mean.set_index(ptype)
    new_col = 'rating_norm_'+base
    df[new_col] = df[[ptype, 'rating']].apply(lambda row : row['rating'] -  mean.loc[int(row[ptype])]['mu'], axis=1)

### Calculer la matrice de corrélation


Dans un système *user-based*, on note $I_u$ l'ensemble des items rensiegnés pour l'utilisateur $u$ et $U_k$ l'ensemble des utilisateurs qui ont notés le film $k$. On note $I_{uv} = I_u \cap I_v$. Pour le *item-based* on utilisera les mêmes notations en intervertissant user et item. On notera également $S_{ui}$ la note normalisée de l'item *i* donnée par l'utilisateur *u*. 


Pour déterminer si deux utilisateurs se ressemblent en termes de goûts, nous utilisons un taux de corrélation sur les avis données. Nous allons comparer quatres taux de corrélations différents. Le premier ```cor()``` calcule le taux de corrélation classique donné par la formule :
$$
cor(u, v) = \frac{\sum_{k \in I_{uv}} s_{uk} s_{vk}}{\sqrt{\sum_{k \in I_{uv}} s_{uk}^2}\sqrt{\sum_{k \in I_{uv}} s_{vk}^2}}
$$

Le taux de corrélation ajusté ```cor_adj()``` permet de ne pas donner trop d'importance aux films populaires que beaucoup de personnes ont vu.
$$
cor\_adj(u, v) = \frac{\sum_{k \in I_{uv}} s_{uk} s_{vk} / U_k}{\sqrt{\sum_{k \in I_{uv}} \frac{s_{uk}^2}{|U_k|}}\sqrt{\sum_{k \in I_{uv}} \frac{s_{vk}^2}{|U_k|}}}
$$

Le taux de correlation calculé par ```cor_dis()``` permet de ne pas donner une correlation trop élevée si les deux utilisateurs n'ont pas donné assez d'avis sur des films en commun. 
$$
cor\_dis(u, v) = cor(u, v) * \frac{min(|I_{uv}|, \beta)}{\beta}
$$

Enfin la fonction ```cor_dis_adj()``` fait un mélange des deux dernières amélioration : il filtre les films trop populaire et n'apporte de l'importance seulement si deux personnes ont données leur avis sur un certain nombre de films.

$$
cor\_dis(u, v) = cor\_adj(u, v) * \frac{min(|I_{uv}|, \beta)}{\beta}
$$

# Big question : on donne quoi comme correlation si su et/ou sv est nul ? J'ai mis 0 par défaut mais bon ...

In [428]:
def cor(df, u, v, Iuv, base):
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    su = df.loc[(df[ptype] == u) & (df[otype].isin(Iuv[otype]))]
    sv = df.loc[(df[ptype] == v) & (df[otype].isin(Iuv[otype]))]
    su = su.rating_norm_user if base == 'user' else su.rating_norm_movie
    sv = sv.rating_norm_user if base == 'user' else sv.rating_norm_movie
    su = np.array(su)
    sv = np.array(sv)
    up = np.dot(su, sv)
    down = math.sqrt(np.dot(su, su) * np.dot(sv, sv))
    if up == 0 or down == 0:
        return 0
    return up/ down

# À adapter encore

In [429]:
def cor_adj(u, v, df, Iuv):
    nb_rat = df.loc[:, ['movieId', 'rating']].groupby(['movieId']).count()
    
    sum_up = 0
    sum_down_u = 0
    sum_down_v = 0
    for movie in Iuv.movieId.unique() :
        suk = df.loc[(df['userId'] == u) & (df['movieId'] == movie), ['rating_norm_'+base]]
        svk = df.loc[(df['userId'] == v) & (df['movieId'] == movie), ['rating_norm_'+base]]
        suk, svk = float(suk), float(svk)
        
        sum_up += suk * svk / nb_rat.at[movie, 'rating']
        sum_down_u += suk**2 /  nb_rat.at[movie, 'rating']
        sum_down_v += svk**2 /  nb_rat.at[movie, 'rating']
    return sum_up / math.sqrt(sum_down_u * sum_down_v)

In [221]:
def cor_dis(u, v, df, Iuv):
    beta = 20
    correlation = cor(u, v, df, Iuv)
    return correlation * min(len(Iuv), beta)/beta

In [222]:
def cor_dis_adj(u, v, df, Iuv):
    beta = 20
    correlation = cor_adj(u, v, df, Iuv)
    return correlation * min(len(Iuv), beta)/beta

# Big question 2 : un gars ([ici](https://www.ethanrosenthal.com/2015/11/02/intro-to-collaborative-filtering/)) fait cette fonction pour calculer la matrice plus rapidement 

In [298]:
def fast_similarity(ratings, kind='user', epsilon=1e-9):
    # epsilon -> small number for handling dived-by-zero errors
    if kind == 'user':
        sim = ratings.dot(ratings.T) + epsilon
    elif kind == 'item':
        sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

Nous construisons maintenant la matrice de correlation. Puisqu'une telle matrice est symétrique, nous avons préféré utiliser une dataframe à deux entrées et ainsi ne stocker la corrélation pour un couple qu'une seule fois. En procédant comme tel, l'ordre dans lequel on désigne un couple peer-peer sera important. Pour faciliter l'accès, nous trions d'abord la datframe `df` pour que les peers soient pris dans l'ordre croissant des id. Ainsi les doubles indices de la dataframe construite auront tous la propriété que le premier indice est strictement inférieur au deuxième. Lors de l'accès à la correlation entre deux peers *u et v* il suffira de les ranger dans le bon ordre.

La fonction de corrélation à utiliser peut être précisée en argument et par défaut la fonction choisie est la corrélation classique.

Nous utilisons également le module logging pour suivre le déroulement du calcul. Celui-ci peut être très long en fonction de la taille des données et en rappelant tous les 10 peers qu

Comme le calcul de la matrice peut-être très long, afin d'voir un suivi du déroulement du calcul, on utilise le module logging. Cette fonctionalité est désactivée par défaut. 

In [430]:
def cor_matrix(cor_fct=cor, base='user', verbose=False):
    '''
    Retourne la dataframe des taux de corrélations entre peers
    '''
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    df.sort_values(by=ptype, inplace=True)
    peers = df.userId.unique() if base == 'user' else df.movieId.unique()
    nb_peers = len(peers)
    
    if verbose:
        logger = logging.getLogger()
        logger.setLevel(logging.INFO)
        logging.info('nb of peers: {}'.format(nb_peers))
    
    correlation = []
    couples = []
    for i in range(nb_peers):
        u = peers[i]
        if verbose and not i % 10 : 
            logging.info('peer nb: {} (id {})'.format(i, u))
        for j in range(i + 1, nb_peers):
            v = peers[j]
            Iu = df.loc[df[ptype] == u, [otype]]
            Iv = df.loc[df[ptype] == v, [otype]]
            Iuv = Iu.join(Iv.set_index(otype), on=otype, how='inner')
            if Iuv.size :
                couples.append((u, v))
                correlation.append(cor_fct(df, u, v, Iuv, base))
    index = pd.MultiIndex.from_tuples(couples, names=['u', 'v'])
    cor = pd.DataFrame(correlation, index=index, columns=['correlation'])
    return cor

In [403]:
def get_cor(u, v, base='user'):
    '''
    Retourne le taux de correlation entre u et v stocké
    '''
    cm = cm_user if base == 'user' else cm_movie
    if u > v :
        u, v = v, u
    index = list(cm.index.values)
    if (u, v) in index :
        return float(cm.loc[(u, v)])
    return float('nan')

### Prédiction

Pour prédire la note donnée par un utilisateur à un film, nous allons faire un moyenne des notes données pour les k peer les plus proches. Dans une approche user-based, on regarde donc les k plus proches utilisateurs, dans une approche item-based, les k films les plus proches. Les plus proches sont ceux qui ont une corrélation la plus élevée. On appelle **p** le peer et **o** l'élément other.

La moyenne effectuée est pondérée par les coefficients de corrélations. On ajoute également la moyenne des notes de **p** pour retrouver une note non normalisée.


$$
\hat{\sigma}_{um} = \mu_u + \frac{\sum_{v \in P_u(m)} s_{vm} \cdot cor(u, v)}{\sum_{v \in P_u(m)} |cor(u,v)|}
$$

# Pourquoi utiliser un parsing fait maison et pas knn non parametré comme tout le monde sur internet ?

In [453]:
def peer_group(k, p, o, base='user'):
    '''
    Retourne les k peers les plus proches de p
    '''
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    
    top = [(float('-inf'), p)] * k
    peers = df.loc[df[ptype] == p, [otype, 'rating_norm_'+base]]
    peers = peers.iloc[0].unique()
    
    for v in peers:
        taux = get_cor(p, v, base)
        if taux > top[-1][0] :
            top += [(taux, v)]
            top.sort(reverse=True)
            top = top[:-1]
    return [t[1] for t in top]

In [456]:
def predict(k, p, o, base='user'):
    '''
    Retourne la prédiction de la note du couple (p, o) utilisant un peer-groupe de taille k
    '''
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    mu = mean_rating(p, base)
    peers = peer_group(k, p, o, base)
    
    sum_up, sum_down = 0, 0
    for friend in peers:
        cor = get_cor(p, friend, base)
        if not math.isnan(cor):
            sfo = df.loc[(df[ptype] == friend) & (df[otype] == o), 'rating_norm_'+base]
            sfo = 0 if len(sfo) <= 0 else float(sfo)
            sum_up += svm * cor
            sum_down += abs(cor)
    sum_down = 1 if sum_down == 0 else sum_down
    return mu + sum_up / sum_down

### Test des fonctions de manière individuelle

In [446]:
df = ratings.loc[(ratings['userId'] <= 30) & (ratings['movieId'] <= 500)]
print(df.shape)
df.head()

(783, 3)


,userId,movieId,rating
0,1,31,2.5
20,2,10,4.0
21,2,17,5.0
22,2,39,5.0
23,2,47,4.0


In [447]:
pid = 4
print(mean_rating(pid, 'movie'))
print(mean_rating(pid, 'user'))

3.0
4.04


In [448]:
normalize('movie')
normalize('user')
df.head()

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

,userId,movieId,rating,rating_norm_movie,rating_norm_user
0,1,31,2.5,-0.250,0.000000
25,2,52,3.0,-0.300,-0.357143
53,2,319,1.0,-1.750,-2.357143
41,2,248,3.0,0.000,-0.357143
22,2,39,5.0,1.375,1.642857


In [449]:
cm_user = cor_matrix(base='user', verbose=True)
cm_user.head()

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
INFO:root:nb of peers: 28
INFO:root:peer nb: 0 (id 1)
INFO:root:peer nb: 10 (id 11)
INFO:root:peer nb: 20 (id 22)


correlation
u v             
1 7     0.000000
2 3    -0.633987
  4     0.541601
  5     0.255686
  7    -0.388913

In [451]:
cm_movie = cor_matrix(base='movie', verbose=True)
cm_movie.head()

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
INFO:root:nb of peers: 260
INFO:root:peer nb: 0 (id 1)
INFO:root:peer nb: 10 (id 11)
INFO:root:peer nb: 20 (id 24)
INFO:root:peer nb: 30 (id 40)
INFO:root:peer nb: 40 (id 60)
INFO:root:peer nb: 50 (id 79)
INFO:root:peer nb: 60 (id 100)
INFO:root:peer nb: 70 (id 123)
INFO:root:peer nb: 80 (id 152)
INFO:root:peer nb: 90 (id 164)
INFO:root:peer nb: 100 (id 177)
INFO:root:peer nb: 110 (id 202)
INFO:root:peer nb: 120 (id 223)
INFO:root:peer nb: 130 (id 235)
INFO:root:peer nb: 140 (id 260)
INFO:root:peer nb: 150 (id 277)
INFO:root:peer nb: 160 (id 306)
INFO:root:peer nb: 170 (id 326)
INFO:root:peer nb: 180 (id 342)
INFO:root:peer nb: 190 (id 356)
INFO:root:peer nb: 200 (id 370)
INFO:root:peer

correlation
u v             
1 2    -0.693334
  3     1.000000
  4     0.000000
  5    -1.000000
  6     0.199381

In [457]:
k = 4
user = 1
movie = 2
p1 = predict(k, user, movie, base='user')
p2 = predict(k, user, movie, base='movie')
print(p)
print(p2)

2.5
3.611111111111111


### Utilisation de toutes les données

In [460]:
df = ratings
t1 = time()
normalize(base='user')
t2 = time()
print(t2 - t1)
t1 = time()
normalize(base='movie')
t2 = time()
print(t2 - t1)
df.head()

26.87325882911682
75.73325705528259


,userId,movieId,rating,rating_norm_user,rating_norm_movie
65377,466,1,5.0,1.203704,1.12753
889,13,1,5.0,1.254717,1.12753
69474,481,1,4.0,-0.137615,0.12753
85228,574,1,4.0,0.488304,0.12753
69096,480,1,5.0,1.001323,1.12753


In [ ]:
t1 = time()
cm_user = cor_matrix(base='user', verbose=True)
t2 = time()
print(t2-t1)
cm_user.head()

INFO:root:nb of peers: 671
INFO:root:peer nb: 0 (id 1)
INFO:root:peer nb: 10 (id 11)
INFO:root:peer nb: 20 (id 21)
INFO:root:peer nb: 30 (id 31)
INFO:root:peer nb: 40 (id 41)
INFO:root:peer nb: 50 (id 51)
INFO:root:peer nb: 60 (id 61)
INFO:root:peer nb: 70 (id 71)
INFO:root:peer nb: 80 (id 81)
INFO:root:peer nb: 90 (id 91)
INFO:root:peer nb: 100 (id 101)
INFO:root:peer nb: 110 (id 111)
INFO:root:peer nb: 120 (id 121)
INFO:root:peer nb: 130 (id 131)


In [ ]:
t1 = time()
cm_movie = cor_matrix(base='movie', verbose=True)
t2 = time()
print(t2-t1)
cm_movie.head()

In [ ]:
k = 4
user = 1
movie = 2
p1 = predict(k, user, movie, base='user')
p2 = predict(k, user, movie, base='movie')
print(p)
print(p2)

### Recommand top 10 movies to user

## Model-based recommendation system

La matrice des notes user-item $R$ est partiellement vide. Ainsi réduire les dimensions de la matrice pourrait améliorer la complexité de nos algorithmes. Une méthode que nous pourrions avoir envie d'utiliser est la décomposision en valeurs singulières : $R = U_{svd} \Sigma V_{svd}$. Cependant cette méthode ne s'applique pas ici étant donné que $R$ n'est pas complète et qu'on a besoin de réaliser des calculs algébriques avec $R$ pour trouver la décomposition.

On considère donc un modèle dans lequel il existe des attributs décrivants les films et les préférences des utilisateurs. La matrice $R$ peut alors être factorisée en produit de deux matrices $U$ et $V$ représentant respectivement les utilisateurs et les items :

$$
R \approx U \times V^T
$$

avec $R \in \mathbb{R}^{n \times m}$ la matrice des notes user-item, $U \in \mathbb{R}^{n \times \ell}$ la matrice des users, $V \in \mathbb{R}^{m \times \ell}$ la matrice des items et $\ell$ le nombre d'attributs. Pour faire un rapprochement avec la SVD, on peut considerer que $U = U_{svd} \Sigma^{1/2}$ et $V = \Sigma^{1/2} V_{svd}$. On note $U_i$ les lignes de $U$ et $V_j$ les lignes de $V$ :
$
U = \left[ \begin{array}{c} U_1 \\ \vdots \\ U_n \end{array} \right]
$ et 
$
V = \left[ \begin{array}{c} V_1 \\ \vdots \\ V_m \end{array} \right]
$
avec $U_i^T, V_j^T \in \mathbb{R^\ell}$.

Dans ce modèle, chaque note $R_{ij}$ associée à un couple user-item $(i, j)$ est le résultat du produit scalaire entre la ligne associée au user $i$ dans $U$ et la ligne associée au item $j$ dans $V$ : $R_{ij} = U_i \cdot V_j^T$. Une fois les matrices $U$ et $V$ apprises, pour prédire une note il suffira de faire le produit scalaire entre les lignes associées.

Trouver $U$ et $I$ revient à minimiser l'erreur entre la note prédite $U_i \cdot V_j^T$ et la véritable note $R_{ij}$. Il s'agit du problème de minimisation suivant, avec $E = \{(i, j) \mbox{ | } R_{ij} \mbox{ connue}\}$ :

$$
(U, V) = argmin_{(U, V)} \sum_{(i, j) \in E} [U_i \cdot V_j^T - R_{ij}]^2
$$

qui est équivalent à:

$$
(U, V) = argmin_{(U, V)} \frac{1}{2}\sum_{(i, j) \in E} [U_i \cdot V_j^T - R_{ij}]^2 + \lambda (\|U_i\|^2 + \|V_j\|^2)
$$

Le terme de droite est un terme régulateur, de paramètre $\lambda$ à ajuster, permettant de prévenir un overfitting.

Pour résoudre ce problème, nous allons utiliser une méthode de descente de gradient.


*Pour résoudre ce problème, on peut utiliser une méthode de descente de gradient. Nous allons ensuite optimiser cette méthode en utilisant d'abord des batch, puis en se réduisant à un problème de moindre carré en fixant alternativement les matrices $U$ et $V$.*

### Descente de gradient (à pas constant)

Dans notre [cours d'optimisation](https://www.ceremade.dauphine.fr/~gontier/enseignement.html) donné par David Gontier, nous avons étudié différentes méthodes de descente de gradient de complexité et d'optimalité différentes. Cependant il nous semble qu'utiliser une version simple à pas $\tau$ constant suffit. Il sera possible de régler cet hyper-paramètre par validation croisée. 

Notre fonction objective est la suivante :
$$
F(U, V) := \sum_{(i, j) \in E} \frac{1}{2}[U_i \cdot V_j^T - R_{ij}]^2 + \frac{\lambda}{2} (\|U_i\|^2 + \|V_j\|^2)
$$

Dans une descente de gradient classique, à chaque itération on met à jour $U$ et $V$ suivant la formule 
$
(U, V) = (U, V) - \tau \nabla F(U, V)
$. Cependant, dans notre cas nous n'allons pas mettre à jour toutes les lignes de $U$ et $V$ simultanément. En effet, puisque la somme dans $F$ ne se fait que sur les couples $(i, j)$ pour lesquels la note est connue, nous allons seulement mettre à jour le couple $(U_i, V_j)$ associé en itérant sur tous les couples $(i, j) \in E$. 

Pour une note $R_{ij}$, on a 
$
\frac{\partial F}{\partial U_i} = V_j^T (U_i \cdot V_j^T - R_{ij}) + \lambda U_i
$
 et 
$
\frac{\partial F}{\partial V_j} = Ui (U_i \cdot V_j^T - R_{ij}) + \lambda V_j
$
donc on peut mettre à jour les lignes $U_i$ et $V_j$ selon les formules 
$$
U_i = Ui - \tau [V_j^T (U_i \cdot V_j^T - R_{ij}) + \lambda U_i]\\
V_j = V_j - \tau [Ui (U_i \cdot V_j^T - R_{ij}) + \lambda V_j]
$$

In [307]:
R = ratings.loc[(ratings['userId'] <= 100) & (ratings['movieId'] <= 100)]

In [308]:
R.head()

,userId,movieId,rating
0,1,31,2.5
20,2,10,4.0
21,2,17,5.0
22,2,39,5.0
23,2,47,4.0


In [309]:
n = len(R.userId.unique())
m = len(R.movieId.unique())
print('n:', n, 'm:', m)

n: 85 m: 76


In [310]:
a = R.movieId.unique()
a.sort()
print(a)
R.userId.unique()

[  1   2   3   4   5   6   7   8   9  10  11  12  14  15  16  17  18  19
  20  21  22  23  24  25  26  28  29  30  31  32  34  35  36  39  40  41
  42  43  44  45  46  47  48  50  52  55  57  58  60  61  62  63  64  65
  68  69  70  72  73  74  76  78  79  81  82  85  86  88  89  92  93  94
  95  97  98 100]


array([  1,   2,   3,   4,   5,   7,   8,   9,  10,  11,  13,  15,  16,
        17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  43,  44,
        46,  47,  48,  49,  50,  55,  56,  57,  58,  59,  60,  61,  63,
        66,  67,  68,  69,  70,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
        94,  95,  96,  97,  98,  99, 100])

Nous remarquons que tous les entiers entre 1 et $n$ (ou $m$) ne sont pas nécessairement utlisés par les id des users (ou des movies). Ceci peut être du au nettoyage effecté par exemple. Puisque nous aimerions utiliser des numpy array dans nos calculs, il va être nécessaire d'avoir la correspondant entre les id et les indices utilisés dans les numpy array (que nous allons appeler rang). Pour cela, utilisons simplement une liste contenant les id et dont l'indice dans la liste d'un id donné correspondra au rang. Pour trouver l'id à partir d'un rang il suffira de faire un simple extraction, pour trouver le rang à partir d'un id on utilisera la méthode `index()`.

In [311]:
user_rank = R.userId.unique().tolist()
movie_rank = R.movieId.unique().tolist()

La matrice $R$ étant vide, nous n'allons pas utiliser de matrice pour la représenter et garderons la dataframe qui ne contient que les notes connues. Nous allons également avoir besoin d'écrire une fonction `get_rat()` qui permet d'accéder à la note d'un couple de rang dans la dataframe des notes. Nous utilisons également une fonction `known()` pour construire l'ensemble $E$.

In [312]:
def get_rat(R, i, j):
    '''
    Retourne la note de rang (i, j) dans la dataframe R
    '''
    uid, mid = user_rank[i], movie_rank[j]
    return float(R.loc[(R['userId'] == uid) & (R['movieId'] == mid), 'rating'])

In [313]:
def known(R):
    '''
    Retourne l'ensemble des indices (i, j) pour lesquels la note est connue dans R
    '''
    ids = set(R.loc[:, ['userId', 'movieId']].itertuples(index=False, name=None))
    E = set(map(lambda t : (user_rank.index(t[0]), movie_rank.index(t[1])), ids ))
    return E

Nous pouvons à présent écrire la fonction résolvant notre problème de minimisation. Remarquons qu'elle modifie les valeurs de $U$ et $V$ en place.

In [332]:
def compute_rmse(U, V, E):
    predicted = np.dot(U, V.T) # the predicted rating matrix

    rmse = 0
    nb_instances = 0
    for (i, j) in E :
        rmse += (predicted[i, j] - get_rat(R, i, j)) ** 2 
        nb_instances += 1
    return np.sqrt(rmse / nb_instances)

In [333]:
def descenteGradient(U, V, tau, tol=1e-3, Niter=100):
    E = known(R)
    last_rmse = 0
    for n in range(Niter):
        
        if not n % 10 :
            print(n, end=' ')
            
        rmse = compute_rmse(U, V, E)
        if abs(rmse - last_rmse) < tol:
            return U, V
        last_rmse = rmse

        for (i, j) in E :
            gradU = V[j].T * (np.dot(U[i], V[j].T) - get_rat(R, i, j)) + lamb * U[i]
            gradV = U[i] * (np.dot(U[i], V[j].T) - get_rat(R, i, j)) + lamb * V[j]
            U[i] = U[i] - tau * gradU
            V[j] = V[j] - tau * gradV
    print("Erreur, l’algorithme n’a pas convergé après", Niter ," itérations")
    return U, V

In [327]:
# hyper-paramètres à tuned
ell = 10
tau = 1/10
lamb = 1/2
U, V = np.random.rand(n, ell), np.random.rand(m, ell)
print(U[:5, :5])
print(V[:5, :5])

[[0.88165623 0.58808305 0.22636837 0.35073326 0.63264633]
 [0.94683774 0.01466973 0.39754509 0.23114582 0.82478488]
 [0.15098563 0.46531669 0.02931014 0.25107286 0.2322986 ]
 [0.54632422 0.08072443 0.94839987 0.93453955 0.4134349 ]
 [0.37386581 0.51396434 0.29181443 0.80846473 0.21604126]]
[[0.62707407 0.83666176 0.44684805 0.04112268 0.98459729]
 [0.62650776 0.45465037 0.70153242 0.36715837 0.38015689]
 [0.91709201 0.4147387  0.07863112 0.52865519 0.37496846]
 [0.43170755 0.69557186 0.85053335 0.98941011 0.37618853]
 [0.46720101 0.33747503 0.22808895 0.294576   0.35026358]]


In [334]:
descenteGradient(U, V, tau)
rmse = compute_rmse(U, V, E)

0 

In [329]:
print(rmse)

0.9998247929748243


In [335]:
def predict(uid, mid):
    user = user_rank[uid]
    movie = movie_rank[mid]
    return np.dot(U[user], V[movie].T)

In [336]:
user = 1
movie = 2

print(predict(user, movie))

2.6639861427882003


### Cross-validation


## Linear model : content-based

On remarque que si $U$ ou $V$ est fixé, la fonction objective devient quadratique. Or nous connaissons des algorithmes efficaces pour minimiser des fonctions quadratiques. De plus, une matrice d'attributs des films peut être donnée puisqu'on connaît certaines informations sur les films.